In [35]:
%load_ext sql
%config SqlMagic.autocommit=False # for engines that do not support autommit

# Question 1 from : https://data36.com/sql-interview-questions-tech-screening-data-analysts/

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
import csv
import pandas as pd

# had issues with tables persisting using in memory 
db_connection_string = 'sqlite:///temp_db.db'

Base = declarative_base()
engine = create_engine( db_connection_string )
Base.metadata.create_all(engine)

engine


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Engine(sqlite:///temp_db.db)

In [36]:
%sql sqlite:///temp_db.db

'Connected: @temp_db.db'

In [37]:
## Advertising / Ad buying SQL exam

# load data into tables as is 
df = pd.read_csv('./data/interview/SQL Interview Samples - accounts.csv')
df.to_sql(con=engine, name='accounts', if_exists='replace')

df = pd.read_csv('./data/interview/SQL Interview Samples - account_managers.csv')
df.to_sql(con=engine, name='account_managers', if_exists='replace')

df = pd.read_csv('./data/interview/SQL Interview Samples - account_domain_deliveries.csv')
df.to_sql(con=engine, name='account_domain_deliveries', if_exists='replace')


In [38]:
%%sql 

-- Question #1
-- list the name of all the retail accounts

SELECT DISTINCT(name)
FROM accounts
WHERE industry = 'Retail'

 * sqlite:///temp_db.db
Done.


name
Jobox
Tables
UNiqlo
AMazon
fake store


In [39]:
%%sql 

-- question #2
-- in alphabetical order show names on accounts that spend more than 15000 in April

select name as account_name, sum(spend)
FROM accounts
JOIN account_domain_deliveries as spend
ON accounts.id = spend.account_id
WHERE date >= "2019-04-01" AND date < "2019-05-01" 
GROUP BY name
HAVING sum(spend)>15000
ORDER BY account_name ASC

 * sqlite:///temp_db.db
Done.


account_name,sum(spend)
Dollie Parton,16388
Jobox,16991
Lamps,16563
Tables,29255
Walmart,15906


In [40]:
%%sql 

-- question #3
-- for each account manager - how much did their combined accunts spend in June 2019? 
-- list the account manager by name and office descending order of total spend

-- STEP 1 join spends with account manager , include name, office, spend
-- STEP 2 - include account managers
-- STEP 3 sum / group / ordre

WITH spent_accounts AS 
( 
    select name as account_name, spend, id, account_id, date
    FROM accounts
    JOIN account_domain_deliveries as spend
    ON accounts.id = spend.account_id
) 

select name, office , sum(spend) as total_spend
FROM spent_accounts
JOIN account_managers
ON spent_accounts.account_id = account_managers.id
WHERE date >= '2019-06-01' and date < '2019-07-01'
GROUP BY name, account_name
ORDER BY total_spend DESC

 * sqlite:///temp_db.db
Done.


name,office,total_spend
Bob,Dubai,13888
Morgan,London,13883
Chi Chi Davonpot,London,12878
Liz Lemon,New York,12483
Jenna Maroni,Dublin,12434
Aja,Dubai,11437
Ru Paul,New York,10606


In [41]:
%%sql 

-- question #4
-- find the names of all retail accounts that spent at least $200 
-- for every day the last seven days of october


-- first combine the spend to the account with account name
WITH accounts_spend AS 
( 
    select name as account_name, spend, id, account_id, date
    FROM accounts
    JOIN account_domain_deliveries as spend
    ON accounts.id = spend.account_id
),

-- then agg total spend by date so it is referenceable
sum_spend_per_day AS 
( 
    select * , sum(spend) as total_spend
    FROM accounts_spend
    GROUP BY account_name, date
) ,

-- how many days in this period did each account meet the minimum spend?
end_oct_filter_spend AS
(
    select  DISTINCT COUNT(account_name) as days_with_min_spend, account_name, total_spend, SUM(total_spend) as period_spend, AVG(total_spend) as avg_spend
    from sum_spend_per_day
    WHERE date > '2019-10-24' AND 
          date <= '2019-10-31' AND
          total_spend > 200
    GROUP BY account_name
)

-- lastly: which accounts had consistent spend overall 7 days?
select account_name, days_with_min_spend, period_spend, avg_spend
FROM end_oct_filter_spend
WHERE days_with_min_spend = 7


 * sqlite:///temp_db.db
Done.


account_name,days_with_min_spend,period_spend,avg_spend
Dollie Parton,7,3607,515.2857142857143
Tables,7,6335,905.0
